In [1]:
import os
import torch
from pm.PM_transformer import PM_Transformer
from stock_data_handle import Stock_Data
import pandas as pd
from utils.tools import dict_to_namespace

In [6]:
import json

# Load the JSON file
with open("model_config\\transformer_config.json", "r") as f:
    config_dict = json.load(f)

args = dict_to_namespace(config_dict)

# Now you can do:
device = torch.device("cuda:0" if args.use_gpu else "cpu")
args.device = device

# And also:
args

namespace(model='Transformer',
          project_name='DOW30',
          root_path='data_dir',
          data_dict={'DOW30': {'dataset_name': 'DOW',
                      'full_stock_path': 'DOW30'},
                     'SSE30': {'dataset_name': 'SSE',
                      'full_stock_path': 'SSE50'},
                     'NASDAQ100': {'dataset_name': 'NAS',
                      'full_stock_path': 'NASQ100'}},
          seq_len=60,
          prediction_len=5,
          rank_alpha=1,
          batch_size=32,
          num_workers=1,
          learning_rate=0.0003,
          train_epochs=50,
          use_multi_gpu=False,
          use_gpu=True,
          enc_in=13,
          dec_in=13,
          c_out=1,
          d_model=256,
          n_heads=4,
          e_layers=2,
          d_layers=1,
          d_ff=512,
          dropout=0.3,
          activation='gelu',
          device=device(type='cuda', index=0))

In [3]:
project_name = args.project_name
stock_data = Stock_Data(dataset_name=args.data_dict[project_name]["dataset_name"], full_stock_path=args.data_dict[project_name]["full_stock_path"],
                            window_size=args.seq_len, root_path=args.root_path, prediction_len=args.prediction_len, scale=True)
    

c:\code\Honour\src\pred-methods\stock_data_handle.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat((df, temp_df))


Generate covariate matrix...
data shape:  (3515, 29, 42)
label shape:  (1, 3515, 29)
Price shape:  (3515, 29)


In [7]:
model_type="Transformer"
print("\n" + "="*40)
print("Testing model type:", model_type)
args.model = model_type
exp = PM_Transformer(args, stock_data)

2025-04-19 20:37:05,414 - PM_Transformer - INFO - Initializing model with log directory: log\pred_DOW30_Transformer
2025-04-19 20:37:05,415 - PM_Transformer - INFO - Log file created at log\pred_DOW30_Transformer\PM_Transformer.log



Testing model type: Transformer
Use GPU: cuda:cuda


In [ ]:
exp.train()

2025-04-19 20:37:09,254 - PM_Transformer - INFO - Starting training for Transformer model
2025-04-19 20:37:09,256 - PM_Transformer - INFO - Training epochs: 50, Steps per epoch: 77
2025-04-19 20:39:42,812 - PM_Transformer - INFO - Epoch 1: Train Loss: 0.0883383, Valid Loss: 0.0017050, Test Loss: 0.0018502
2025-04-19 20:39:42,815 - PM_Transformer - INFO - mirr_top1/train: 0.0030489321782967292
2025-04-19 20:39:42,815 - PM_Transformer - INFO - rank_ic/train: -0.0076045882438456934
2025-04-19 20:39:42,816 - PM_Transformer - INFO - mirr_top1/valid: 0.0022016080177348595
2025-04-19 20:39:42,817 - PM_Transformer - INFO - rank_ic/valid: -0.006717853286147096
2025-04-19 20:39:42,817 - PM_Transformer - INFO - mirr_top1/test: -0.0011788445214430492
2025-04-19 20:39:42,818 - PM_Transformer - INFO - rank_ic/test: -0.007828269673424676
2025-04-19 20:39:42,842 - PM_Transformer - INFO - Model checkpoint saved: ./checkpoints/transformer\checkpoint_1.pth
2025-04-19 20:39:42,843 - PM_Transformer - INFO 

In [6]:
best_model_path = os.path.join("checkpoints\\transformer", f"checkpoint_{1}.pth")
exp.model.load_state_dict(torch.load(best_model_path))

<All keys matched successfully>

In [7]:
exp.backtest(topk=1,holding_period=5)

2025-04-19 02:09:47,553 - PM_Transformer - INFO - 
── Back‑test ─────────────────────────────────────────
2025-04-19 02:09:47,554 - PM_Transformer - INFO - top‑1 | hold 5‑d
2025-04-19 02:09:47,554 - PM_Transformer - INFO - Sharpe       : -0.1591
2025-04-19 02:09:47,556 - PM_Transformer - INFO - Sortino      : -0.2490
2025-04-19 02:09:47,556 - PM_Transformer - INFO - Max DD       : -0.5752
2025-04-19 02:09:47,557 - PM_Transformer - INFO - Ann. Return  : -0.1204
2025-04-19 02:09:47,557 - PM_Transformer - INFO - Total Return : -0.3177
2025-04-19 02:09:47,558 - PM_Transformer - INFO - 
Benchmark (Market Avg):
2025-04-19 02:09:47,558 - PM_Transformer - INFO - Sharpe       : 0.4685
2025-04-19 02:09:47,559 - PM_Transformer - INFO - Sortino      : 0.6832
2025-04-19 02:09:47,560 - PM_Transformer - INFO - Max DD       : -0.2156
2025-04-19 02:09:47,560 - PM_Transformer - INFO - Ann. Return  : 0.0599
2025-04-19 02:09:47,561 - PM_Transformer - INFO - Total Return : 0.1892
2025-04-19 02:09:47,561 - 

In [8]:
exp.backtest(topk=10,holding_period=5)

2025-04-19 02:09:52,908 - PM_Transformer - INFO - 
── Back‑test ─────────────────────────────────────────
2025-04-19 02:09:52,908 - PM_Transformer - INFO - top‑10 | hold 5‑d
2025-04-19 02:09:52,909 - PM_Transformer - INFO - Sharpe       : 0.2468
2025-04-19 02:09:52,909 - PM_Transformer - INFO - Sortino      : 0.3629
2025-04-19 02:09:52,910 - PM_Transformer - INFO - Max DD       : -0.2725
2025-04-19 02:09:52,910 - PM_Transformer - INFO - Ann. Return  : 0.0280
2025-04-19 02:09:52,911 - PM_Transformer - INFO - Total Return : 0.0858
2025-04-19 02:09:52,911 - PM_Transformer - INFO - 
Benchmark (Market Avg):
2025-04-19 02:09:52,912 - PM_Transformer - INFO - Sharpe       : 0.4685
2025-04-19 02:09:52,913 - PM_Transformer - INFO - Sortino      : 0.6832
2025-04-19 02:09:52,913 - PM_Transformer - INFO - Max DD       : -0.2156
2025-04-19 02:09:52,914 - PM_Transformer - INFO - Ann. Return  : 0.0599
2025-04-19 02:09:52,914 - PM_Transformer - INFO - Total Return : 0.1892
2025-04-19 02:09:52,915 - PM_

# LSTM

In [7]:
import os
import torch
from pm.PM_lstm import PM_LSTM
from stock_data_handle import Stock_Data
import pandas as pd
from utils.tools import dict_to_namespace

In [8]:
import json

# Load the JSON file
with open("model_config\\alstm_config.json", "r") as f:
    config_dict = json.load(f)

args = dict_to_namespace(config_dict)

# Now you can do:
device = torch.device("cuda:0" if args.use_gpu else "cpu")
args.device = device

# And also:
args

namespace(model='ALSTM',
          project_name='DOW30',
          root_path='data_dir',
          data_dict={'DOW30': {'dataset_name': 'DOW',
                      'full_stock_path': 'DOW30'},
                     'SSE30': {'dataset_name': 'SSE',
                      'full_stock_path': 'SSE50'},
                     'NASDAQ100': {'dataset_name': 'NAS',
                      'full_stock_path': 'NASQ100'}},
          seq_len=20,
          prediction_len=1,
          rank_alpha=3,
          batch_size=32,
          num_workers=1,
          learning_rate=0.0001,
          train_epochs=30,
          use_multi_gpu=False,
          use_gpu=True,
          model_config={'input_size': 5,
                        'hidden_size': 32,
                        'num_layers': 1,
                        'dropout': 0.3,
                        'bidirectional': False,
                        'attention': True,
                        'noise_level': 0.02,
                        'd_ff': 64,
              

In [9]:
project_name = args.project_name
stock_data = Stock_Data(dataset_name=args.data_dict[project_name]["dataset_name"], full_stock_path=args.data_dict[project_name]["full_stock_path"],
                            window_size=args.seq_len, root_path=args.root_path, prediction_len=args.prediction_len, scale=True)
    

c:\code\Honour\src\pred-methods\stock_data_handle.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat((df, temp_df))


Generate covariate matrix...
data shape:  (3519, 29, 42)
label shape:  (1, 3519, 29)
Price shape:  (3519, 29)


In [10]:
model_type="ALSTM"
print("\n" + "="*40)
print("Testing model type:", model_type)
args.model = model_type
exp = PM_LSTM(args, stock_data)

c:\Users\gaomi\anaconda3\envs\RL\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(
2025-04-19 05:48:11,160 - PM_ALSTM - INFO - Initializing model with log directory: log\pred_DOW30_ALSTM
2025-04-19 05:48:11,161 - PM_ALSTM - INFO - Log file created at log\pred_DOW30_ALSTM\PM_ALSTM.log



Testing model type: ALSTM
Use GPU: cuda:cuda


In [11]:
exp.train()

2025-04-19 05:48:11,172 - PM_ALSTM - INFO - LSTM training start: epochs=30, batch_size=32
2025-04-19 05:48:23,399 - PM_ALSTM - INFO - Epoch 1: Train 0.000428, Valid 0.000309, Test 0.000348
2025-04-19 05:48:23,402 - PM_ALSTM - INFO - mirr_top1/train: 0.0008787806198084488
2025-04-19 05:48:23,405 - PM_ALSTM - INFO - rank_ic/train: 0.0028788818563809594
2025-04-19 05:48:23,407 - PM_ALSTM - INFO - mirr_top1/valid: 0.00015890973356916854
2025-04-19 05:48:23,409 - PM_ALSTM - INFO - rank_ic/valid: -0.012730247140189171
2025-04-19 05:48:23,411 - PM_ALSTM - INFO - mirr_top1/test: 0.0003213355722393663
2025-04-19 05:48:23,412 - PM_ALSTM - INFO - rank_ic/test: -0.004561787599556212
2025-04-19 05:48:23,415 - PM_ALSTM - INFO - Best updated at epoch 1, Val 0.000309
2025-04-19 05:48:35,197 - PM_ALSTM - INFO - Epoch 2: Train 0.000306, Valid 0.000284, Test 0.000308
2025-04-19 05:48:35,198 - PM_ALSTM - INFO - mirr_top1/train: 0.0006342994928649531
2025-04-19 05:48:35,199 - PM_ALSTM - INFO - rank_ic/trai

LSTM(
  (input_drop): Dropout(p=0.3, inplace=False)
  (rnn): LSTM(5, 32, batch_first=True, dropout=0.3)
  (W): Linear(in_features=32, out_features=32, bias=True)
  (u): Linear(in_features=32, out_features=1, bias=False)
  (projection): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): GELU(approximate='none')
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [12]:
exp.backtest(topk=5,holding_period=3)

2025-04-19 05:56:37,464 - PM_ALSTM - INFO - 
── Back-test Results ────────────────────────────
2025-04-19 05:56:37,465 - PM_ALSTM - INFO - top-5, hold 3: Sharpe -0.1933, Sortino -0.3078, MaxDD -0.2578, AnnRet -0.0584
2025-04-19 05:56:37,465 - PM_ALSTM - INFO - Benchmark: Sharpe 0.5801, Sortino 0.8285, MaxDD -0.1941, AnnRet 0.0769
2025-04-19 05:56:37,466 - PM_ALSTM - INFO - Info Ratio: -1.1168
2025-04-19 05:56:37,472 - PM_ALSTM - INFO - Saved backtest CSV to log\backtest_results\ALSTM\perf_top5_hold3.csv
